In [2]:
import numpy as np
import pandas as pd
pd.pandas.set_option('display.max_columns', None)

In [3]:
## Sample connection
from sqlalchemy import create_engine

host = "creative-analytics-db.csmwsaq07qyi.us-east-1.rds.amazonaws.com"
port = 3306
user = "admin"
password = "creativeanalytics"

engine = create_engine(f"mysql://{user}:{password}@{host}:{port}/creative_analytics")

In [3]:
def get_object_spatial_position(bbox):
    """
       Function that returns the spacial position of object

               Parameters:
                       bbox (list): bounding box

               Returns:
                       position (str): position value
    """
    # compute center coordinates
    x = bbox[0] + (bbox[2] / 2)
    y = bbox[1] + (bbox[3] / 2)

    left = x < 0.4
    center_x = 0.4 <= x <= 0.65
    right = x > 0.65
    top = y < 0.4
    center_y = 0.4 <= y <= 0.65
    bottom = y > 0.65

    if top and left:
        return "top_left"
    elif top and center_x:
        return "top_center"
    elif top and right:
        return "top_right"
    elif center_y and left:
        return "center_left"
    elif center_y and center_y:
        return "center"
    elif center_y and right:
        return "center_right"
    elif bottom and left:
        return "bottom_left"
    elif bottom and center_x:
        return "bottom_center"
    elif bottom and right:
        return "bottom_right"

In [4]:
def get_object_temporal_position(startFrame, endFrame, totalFrame):
    ''' 
    beginning: first 33% of video.
    middle: middle 33% of video.
    end: last 33% of video.
    
    '''
    start_beg_frame = False
    start_mid_frame = False
    start_end_frame = False

    end_beg_frame = False
    end_mid_frame = False
    end_end_frame = False

    level_1 = 33*totalFrame/100
    level_2 = 66*totalFrame/100

    if startFrame <= level_1:
        start_beg_frame = True
    elif startFrame > level_1 and startFrame <= level_2:
        start_mid_frame = True
    else:
        start_end_frame = True

    if endFrame <= level_1:
        end_beg_frame = True
    elif endFrame > level_1 and endFrame <= level_2:
        end_mid_frame = True
    else:
        end_end_frame = True

    if start_beg_frame and end_beg_frame:
        return "beginning"
    elif start_mid_frame and end_mid_frame:
        return "middle"
    elif start_end_frame and end_end_frame:
        return "end"
    elif start_beg_frame and end_mid_frame:
        return "beginning_to_middle"
    elif start_beg_frame and end_end_frame:
        return "beginning_to_end"
    elif start_mid_frame and end_end_frame:
        return "middle_to_end"

In [4]:
pd.read_sql("SHOW TABLES", con=engine)

,Tables_in_creative_analytics
0,0117_hair_us_youtube_engagement
1,20221220_hair_us_branding_metrics
2,20221220_hair_us_tiktok_engagement
3,20221220_hair_us_tiktok_tags
4,20223012_CORRELATIONS_aggregate_asset_data
5,20223012_CORRELATIONS_asset_data
6,20223012_CORRELATIONS_audience_data
7,20223012_CORRELATIONS_framewise_branding_metrics
8,20223012_CORRELATIONS_other_metrics
9,20230102_collaborativeads_ad_adsets_pivot


In [6]:
table = '20230123_BAYER_UMBRELLA_BRAND_competitor_assets_url'
df = pd.read_sql(f"SELECT * FROM {table}", con=engine)

In [6]:
#--------TABLE 1: annotation of manual tagging------------------------
table = '20230123_BAYER_UMBRELLA_BRAND_logo_annotations'
df_annotation = pd.read_sql(f"SELECT * FROM {table}", con=engine)

#--------TABLE 2: branding metrics-----------------------------------
table = '20230123_BAYER_UMBRELLA_BRAND_branding_metrics'
df_branding = pd.read_sql(f"SELECT * FROM {table}", con=engine)

#--------TABLE 3: performance data-----------------------------------
table = '20230123_BAYER_UMBRELLA_BRAND_ad_performance_data'
df_performance = pd.read_sql(f"SELECT * FROM {table}", con=engine)

#--------TABLE 4: meta data-----------------------------------
table = '20230123_BAYER_UMBRELLA_BRAND_ad_metadata'
df_metadata = pd.read_sql(f"SELECT * FROM {table}", con=engine)

#--------TABLE 5: tags from tagging system-----------------------------------
table = '20230123_BAYER_UMBRELLA_BRAND_tags'
df_tags = pd.read_sql(f"SELECT * FROM {table}", con=engine)
df_tags.to_csv('data/tags.csv', index=False)

In [7]:
df_annotation['logo_position'] = 0
df_annotation['logo_time'] = 0

for i in range(len(df_annotation)):
    bbox = [
        df_annotation.annotation_bboxCoordTopLeftX.values[i],
        df_annotation.annotation_bboxCoordTopLeftY.values[i],
        df_annotation.annotation_bboxCoordBottomRightX.values[i],
        df_annotation.annotation_bboxCoordBottomRightY.values[i]
        ]
    df_annotation.loc[i, 'logo_position'] = get_object_spatial_position(bbox)

    startFrame = df_annotation.annotation_startFrame.values[i]
    endFrame = df_annotation.annotation_endFrame.values[i]
    totalFrame = df_annotation.frameCount.values[i]
    df_annotation.loc[i, 'logo_time'] = get_object_temporal_position(startFrame, endFrame, totalFrame)

In [33]:
df_annotation.to_csv('data/annotation.csv', index=False)
df_branding.to_csv('data/branding_metrics.csv', index=False)
df_metadata.to_csv('data/metadata.csv', index=False)
df_performance.to_csv('data/performance_metrics.csv', index=False)

In [8]:
df_annotation.annotation_name.unique()

array(['Logo: primary', 'Logo: in pack', 'Logo: always on',
       'Company Logo'], dtype=object)

In [10]:
df_annotation.loc[df_annotation.brandName=='Actron Argentina'].assetId.nunique()

129

In [30]:
df_performance.loc[(df_performance.carrousel_position.isna()==True) | (df_performance.carrousel_position=='1')].ad_id.nunique()

361

In [48]:
df_performance.carrousel_position.unique()

array([None, '1', '5', '4', '0', '3', '2', '6', '8', '7'], dtype=object)